In [2]:
# Get the request from wikipedia
import requests
import lxml.html as lh
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
doc = lh.fromstring(website_url.content)
tr_elements = doc.xpath('//tr')

In [4]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')

col = []
i = 0
for t in tr_elements[0]:
    i+=1
    name = t.text_content()
    #print('%d:"%s"'%(i,name))
    col.append((name,[]))
    
del col[-1]
col.append(("Neighborhood",[]))
col[0] = ("PostalCode",[])
col

[('PostalCode', []), ('Borough', []), ('Neighborhood', [])]

In [6]:
for j in range(1,len(tr_elements)):
    T = tr_elements[j]
    if len(T) != 3:
        break
    i = 0
    for t in T.iterchildren():
        data = t.text_content()
        if i > 0:
            try:
                data = int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [9]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [11]:
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)
df = df.drop(df[(df.Borough == "Not assigned")].index)
df = df.reset_index()
del df['index']
df['Neighborhood'] = df['Neighborhood'].map(lambda x: str(x)[:-1])

In [101]:
postalCodes = df['PostalCode'].unique()
new_df = pd.DataFrame()
for postalCode in postalCodes:
    rows = df[df['PostalCode'] == postalCode]
    if len(rows) == 1:
        new_df = new_df.append(rows)
    else:
        concat_hood = ""
        for row in rows.iterrows():
            concat_hood += row[1]['Neighborhood'] + ", "
        rows.iloc[0]['Neighborhood'] = concat_hood[:-2]
        new_df = new_df.append(rows.iloc[0])
        
a = new_df[(new_df.Neighborhood == "Not assigned")].index
new_df.Neighborhood[a] = new_df.Borough[a]
postalcode = new_df['PostalCode'].tolist()
new_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park


In [51]:
postal_code = pd.read_csv('https://cocl.us/Geospatial_data')

In [110]:
postal_code.head()
post_code = postal_code.values.tolist()

In [111]:
latitude = [] 
longitude = []
pc = []
for i in range(0,len(postalcode)):
    for j in range(0,len(post_code)):
        if postalcode[i] == post_code[j][0]:
            pc.append(postalcode[i])
            latitude.append(post_code[j][1])
            longitude.append(post_code[j][2])

In [112]:
lat_long = pd.DataFrame({'PostalCode':pc,'Latitude': latitude, 'Longitude': longitude})

In [113]:
complete_df = new_df.join(lat_long.set_index('PostalCode'), on='PostalCode')
complete_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
